# 배경 제거
배경을 제거하고자 하는 전경 주위에 사각형 박스를 그리고 그랩컷(grabCut) 알고리즘을 적용하여 배경을
제거한다.
grabCut의 경우 잘 작동하더라도 여전히 이미지에 제거하지 못한 배경이 발생할 수 있다.
이렇게 제거 되지 못한 부분은 다시 적용하여 제거할 수 있지만 실전에서 수 천장의 이미지를 수동으로 고
치는 것은 불가능한 일이므로 머신러닝을 적용한다거나 할 때도 일부러 noise를 적용하는 것처럼 일부 배
경이 남아있는 것을 수용하는 것이 좋다.

In [1]:
# 배경 제거
import cv2
from utils import image_show
import numpy as np

In [2]:
# 이미지 읽기
image_path = "./test.jpg"

image = cv2.imread(image_path)
print(image.shape)

(500, 500, 3)


In [3]:
# 사각형 좌표 : 사각점의 x y  넓이 높이
rectangle = (0, 0, 400, 400)

In [4]:
# 초기 마스크 생성
mask = np.zeros(image.shape[:2], np.uint8)

In [5]:
# grabCut 에 사용할 임시 배열 생성
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

In [6]:
# grabCut 실행
# image -> 원본 이미지 , bgdModel -> 배경을 위한 임시 배열 fgdModel-> 전경배경 ,
# 5- > 반복횟수 cv2.GC_INIT_WITH_RECT -> 사각형 초기화
cv2.grabCut(image, mask, rectangle, bgdModel,
            fgdModel, 5, cv2.GC_INIT_WITH_RECT)

(array([[2, 2, 2, ..., 0, 0, 0],
        [2, 2, 2, ..., 0, 0, 0],
        [2, 2, 2, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[8.55372972e-01, 9.58700845e-03, 4.53700486e-02, 8.39332109e-02,
         5.73675994e-03, 2.42000000e+02, 2.42000000e+02, 2.42000000e+02,
         4.97669735e+01, 5.06035673e+01, 1.84570196e+02, 1.83115015e+02,
         1.83325228e+02, 1.84685957e+02, 2.24924093e+02, 2.24923764e+02,
         2.24923633e+02, 3.12019231e+01, 2.49278846e+01, 1.97993269e+02,
         1.00000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e-02, 6.38332925e+02, 6.54363317e+02, 7.52019156e+01,
         6.54363317e+02, 6.80205902e+02, 8.95862283e+01, 7.52019156e+01,
         8.95862283e+01, 2.18228962e+02, 1.23991625e+03, 1.23788806e+03,
         1.22666566e+03, 1.23788806e+03, 1.2

In [7]:
# 배경인 곳은 0 그 외에는 1로 설정한 마스크 생성
mask_2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

In [8]:
# 이미지에 새로운 마스크 곱해서 -> 배경 제외
image_rgb_nobg = image * mask_2[:, :, np.newaxis]
image_show(image_rgb_nobg)
cv2.imwrite("01.Remove Background.png",image_rgb_nobg)

True

# 설명 

위에서 먼저 전경이 들어있는 영역 주위를 사각형으로 표시 하였는데, grabCut은 이 사각형 밖에 있는 모든 것이 배경이 라고 가정하고 이 정보를 사용하여 사각형 안에 있는 배경을 찾는다